# AI-medicine 연습

In [1]:
from dotenv import load_dotenv
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_community.chains.pebblo_retrieval.enforcement_filters import PINECONE
from langchain_core.messages import HumanMessage

load_dotenv()

True

## 데이터 로드 / 병합
- 의약품  `medicine_docs`
- 의약외품 `sanitary_aid_docs`
- 총 Document 개수: 56786

In [2]:
from langchain.schema import Document
import pandas as pd
import os

# !pip install openpyxl

# 데이터를 저장할 리스트
medicine_docs = []
sanitary_aid_docs = []

# 엑셀 파일 경로
medicine_info_paths = ['../data/medicine_info_0.xlsx', '../data/medicine_info_1.xlsx', '../data/medicine_info_2.xlsx', '../data/medicine_info_3.xlsx']
sanitary_aid_info_paths = ['../data/sanitary_aid_info_0.xlsx', '../data/sanitary_aid_info_1.xlsx']

# 파일 경로 체크
print("파일 리스트:", medicine_info_paths, sanitary_aid_info_paths)
for path in medicine_info_paths + sanitary_aid_info_paths:
    print(f"Checking path: {path}")
    print(f"Exists: {os.path.exists(path)}")


# 엑셀 파일 리드 >> Document 변환
for path in medicine_info_paths:
    df = pd.read_excel(path) 
    # 각 행을 Document로 변환
    for _, row in df.iterrows():
        content = row.to_dict() # 행 데이터 >> dict
        medicine_docs.append(Document(page_content=str(content)))

for path in sanitary_aid_info_paths:
    df = pd.read_excel(path)
    for _, row in df.iterrows():
        content = row.to_dict()
        sanitary_aid_docs.append(Document(page_content=str(content)))

# 결과 확인
print(f"의약품 Document 개수: {len(medicine_docs)}")
print(f"의약외품 Document 개수: {len(sanitary_aid_docs)}")
print(f"의약품 첫 번째 Document 내용: {medicine_docs[0].page_content}")
print(f"의약외품 첫 번째 Document 내용: {sanitary_aid_docs[0].page_content}")


파일 리스트: ['../data/medicine_info_0.xlsx', '../data/medicine_info_1.xlsx', '../data/medicine_info_2.xlsx', '../data/medicine_info_3.xlsx'] ['../data/sanitary_aid_info_0.xlsx', '../data/sanitary_aid_info_1.xlsx']
Checking path: ../data/medicine_info_0.xlsx
Exists: True
Checking path: ../data/medicine_info_1.xlsx
Exists: True
Checking path: ../data/medicine_info_2.xlsx
Exists: True
Checking path: ../data/medicine_info_3.xlsx
Exists: True
Checking path: ../data/sanitary_aid_info_0.xlsx
Exists: True
Checking path: ../data/sanitary_aid_info_1.xlsx
Exists: True
의약품 Document 개수: 36983
의약외품 Document 개수: 19803
의약품 첫 번째 Document 내용: {'제품명': '(주)창신의료용산소', '제품영문명': 'Chang-Shin Medical Oxygen', '업체명': '(주)창신', '업체영문명': 'Chang-Shin Co.,LTD', '품목구분': '의약품', '주성분': '산소', '첨가제': nan, '전문의약품': '전문의약품', '완제/원료': '완제의약품', '마약구분': nan, '모양': nan, '색상': nan, '제형': nan, '장축': nan, '단축': nan, '주성분영문': 'Oxygen'}
의약외품 첫 번째 Document 내용: {'제품명': '#4500제이라스틱', '업체명': '(주)곰스포츠', '업체영문명': nan, '품목구분': '의약외품', '주성분': n

In [3]:
documents = medicine_docs + sanitary_aid_docs
print(len(documents))

56786


## 임베딩 모델 생성

: RAG를 구축하기 위한 임베딩 모델 생성
* 문서 내용을 벡터화하여 검색 및 검색된 정보를 바탕으로 답변 생성에 사용

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

## Pinecone Vector DB 클라이언트 생성
- 임베딩 벡터를 벡터 저장소에 저장하고, 이를 효율적으로 검색하기 위해 사용
- Pinecone은 벡터 검색을 위한 클라우드 기반 서비스 / 매우 빠르고 효율적으로 벡터 저장 & 검색 시스템

In [ ]:
!pip install langchain-pinecone

In [5]:
# batch로 documents 밀어넣기
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')


def batch_documents(documents, batch_size=5000):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i+batch_size]
        
batch_size = 5000
for batch in batch_documents(documents, batch_size):
    PineconeVectorStore.from_documents(
        batch,
        embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key = PINECONE_API_KEY
    )

In [ ]:
# # Pinecone 클라이언트
# 
# from langchain_pinecone import PineconeVectorStore
# import os
# 
# PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
# PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
# PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
# 
# # document 밀어넣기
# vector_db = PineconeVectorStore.from_documents(
#     documents,
#     embeddings,
#     index_name=PINECONE_INDEX_NAME,
#     namespace=PINECONE_NAMESPACE,
#     pinecone_api_key = PINECONE_API_KEY
# )

In [ ]:
os.getenv('PINECONE_INDEX_NAME')

## Pinecone 유사도 검색

In [ ]:
# # 클라이언트 객체
# vector_db = PineconeVectorStore(
#     embeddings=embeddings.
#     index_name=PINECONE_INDEX_NAME,
#     namespace=PINECONE_NAMESPACE,
#     pinecone_api_key = PINECONE_API_KEY
# )
#

## 서비스 프롬프트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

### 1. 약물 복용 가이드 서비스
**목적**: 사용자가 현재 복용 중인 약물에 대한 **안전한 복용 가이드라인**을 제공.

**주요 기능**:

- **복용 중인 약물 안전성 검토**:
    - 사용자가 입력한 약물 리스트의 성분 분석.
    - 성분 간 **상호작용 경고** 및 **부작용 안내**.
- **복용 가이드라인 제공**:
    - 연령, 건강 상태(임신, 노인 등) 기반의 권장 용량.
    - 복용 시간 간격 및 식사 여부 관련 주의사항.


In [ ]:
def guide_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        
        '''),
        ('user', '제가 현재 감기약 A와 두통약 B를 복용 중인데, 제대로 먹고 있는지 확인하고 싶어요.')
    ])
    
    prompt += HumanMessagePromptTemplate.from_messages([query])
    model = ChatOpenAI(model='gpt-4o', temperature = 0)
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    
    return chain

### 2. 의약품 / 의약외품 추천 서비스 (made in korea)
**목적**: 사용자의 **개인 맞춤형 의약품** 또는 의약외품을 추천.

**주요 기능**:

- **사용자 입력**: 증상, 선호 성분(포함/제외), 연령, 건강 상태.
- **맞춤 추천**:
    - 입력된 조건에 부합하는 **한국산 의약품/의약외품** 추천.
    - 증상 완화에 적합한 **대체 의약품** 옵션 제시.
- **제품 비교**:
    - 추천 제품 간 주요 성분 및 가격 비교.
    - 사용 후기 및 평판 기반의 제품 선택 지원.

In [ ]:
def recommend_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        
        '''),
        # ('user', '임산부인데, 감기에 걸렸어요. 안전하게 먹을 수 있는 약 추천해 주세요.')
    ])

### 3. 약물 안전성 검증 서비스
**목적**: 새로운 약물 또는 의약외품 복용 전, 안전성을 확인.

**주요 기능**:

- **약물 입력 및 검증**:
    - 새롭게 복용하려는 약물의 성분 분석.
    - 현재 복용 중인 약물과의 **상호작용 위험** 평가.
- **부작용 및 위험 경고**:
    - 성분 간 부작용 가능성.
    - 사용자 건강 상태(임신, 만성질환 등)에 따른 **주의사항**.
- **안전 대체제 추천**:
    - 위험성 경고 시, 더 안전한 대체 의약품 제안.


In [ ]:
def check_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        
        '''),
        # ('user', '생리통 진통제 A를 복용 중인데, 두통약 E를 추가로 먹어도 될까요?')
    ])